In [ ]:
# Python Notebook to explore different recommender systems, with recommendation of songs based on Spotify data.
# Since we don't have users' data, we will focus on Content-Based Filtering

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import sklearn as skl # ML algorithms and processing
from sklearn import metrics
import json
import math
import random

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Data Read
tracks = pd.read_csv('/kaggle/input/spotify-dataset-19212020-160k-tracks/tracks.csv')

#artists = pd.read_csv('/kaggle/input/spotify-dataset-19212020-160k-tracks/artists.csv')

#genres = pd.read_csv('/kaggle/input/spotify-dataset-19212020-160k-tracks/data_by_genres_o.csv')

**NULL CHECKING AND BASIC INVESTIGATION**

We will now do introductory investigation into each of the files we have read in, mainly just looking for data anomalies or things to clean. More exploratory will come later in the notebook.

In [ ]:
# Tracks investigation
tracks.info()
tracks.describe()

# drop NULL
tracks.dropna(inplace = True)
len(tracks)

Based on the investigative code in the above cell, it seems only the "name" column has any NULL values, and very few at that. For presentability purposes, these rows will be dropped. None of the other columns have NULL values, and they are the ones on which most of our system will be built around, so dropping the NULL rows should have very little effect.

Off of initial thoughts, a fair bit of this will be useful in recommmending tracks. Things like the loudness of a track and the danceability for example play a role in determining whether or not I personally will enjoy a track, so these will be important should we decide to recommend.

**DATA EXPLORATORY**

In [ ]:
# tracks

# heatmap
ax = sns.heatmap(tracks[['acousticness', 'danceability', 'duration_ms', 'energy',
                         'instrumentalness', 'liveness', 'loudness', 'speechiness',
                         'tempo', 'valence', 'popularity']].corr())

Above is a heatmap expressing the correlation between various numeric columns in our track dataset. The majority of the correlation values between columns hovers around -0.2 to 0.2. 

Some of the notable characteristics are, thus, those with a larger magnitude of correlation, whether negative or positive. Among these, we see a track's acoustiness is negatively correlated with its loudness, energy, and, interestingly, its popularity. A song's energy is very positively correlated with its loudness, tempo, valence, and popularity. From this, we can determine a song with a lot of energy and loudness is more likely to be popular, while lower tempo, more acoustic songs tend to be less popular.

These sorts of trends may hint at cluster analysis being useful to identify different "types" of tracks, possibly with the goal of attributing characteristics and genres to the clusters.

In [ ]:
# create figure for a 3x3 grid to analyze histograms
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(3, 3, sharey = True, figsize=(10,10))
#fig = plt.figure(figsize=(1, 1))

axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]
features = ['acousticness', 'danceability', 'duration_ms', 'energy', 
            'instrumentalness', 'liveness', 'loudness', 'speechiness', 'popularity']

for ax,feat in zip(axes, features):
    ax.hist(x = tracks[feat], bins = 40)
    ax.set(xlabel=feat, ylabel='# of tracks')

# examine popularity
#plt.hist(x = tracks['popularity'], bins = 50)
#plt.xlabel('Popularity')
#plt.ylabel('# of Tracks')

When a number of the relevant features are analyzed, it appears that the distributions of the traits across all tracks are rarely normal. That is not necessarily surprising, given tracks are typically of a certain length, and generally should be a certain level of loudness if the artist actually wants the listener to hear the music.

Some more interesting things we can see are the approximate normality of the energy feature, and given its comparatively high correlation with a number of features it follows that those as well should be approximately normal. Yet, popularity doesn't follow precisely. Another is how left-skewed a ton of these features are. The liveness, instrumentalness, and speechiness features are all noticeably skewed to values of low magnitude.

There appears to be a minor left-skew to track popularity as well, meaning a ton of tracks are not very popular at all, with a bit higher density appearing in the range of 20-40. Granted, this makes sense, but it could have a noticeable effect if, moving forward, the goal of our system is to find esoteric tracks or deep-cuts. A slight shift in weight to make popularity more or less important in deciding the track recommended could play a very noticeable role in producing different track recommendations.


**CLUSTER ANALYSIS**

In [ ]:
# maybe some dimension reduction could help, PCA?

from sklearn import cluster as cs

# random data selection
subset = tracks[['acousticness', 'danceability', 'duration_ms', 'energy',
                         'instrumentalness', 'liveness', 'loudness', 'speechiness',
                         'tempo', 'valence', 'popularity']]
# take a (hopefully) representative sample of the dataset, since clustering with 600k points is a bit long
clusterdata = subset.sample(n = 100000)

# evaluation vectors
inertia = []
silhouettescores = []
# would like to test more, but the time committment for training and evaluating is not worth it
clusters = [2,4,6,8]

# K-Means & diagnostics
for c in clusters:
    # create our object and fit
    kmeans_cluster = cs.MiniBatchKMeans(n_clusters = c, batch_size = 100) # batch size = 100 is default
    kmeans_cluster.fit(clusterdata)
    # evaluate based on inertia and silhoutte score
    inertia.append(kmeans_cluster.inertia_)
    silhouettescores.append(metrics.silhouette_score(clusterdata, kmeans_cluster.labels_, metric = 'euclidean'))

In [ ]:
# plotting the results
# inertia plot
plt.plot(clusters, inertia)
plt.xlabel("Cluster #")
plt.ylabel("Inertia")
plt.show()

# silhouette score plot
plt.plot(clusters, silhouettescores)
plt.xlabel("Cluster #")
plt.ylabel("Silhouette Scores")
plt.show()

We choose a cluster value of 8 in this run, as we can see the inertia minimize at 8, as well as an elbow point after 4 clusters in the silhouette score. This indicates that the formation quality (relative density and separation) of these clusters takes a notable hit when moving from 2 to 4 and above, while inertia steadily improves.

Of course, something to consider carefully is that the cluster algorithm is ran on a random sample of the data about 1/6 of the total size. It follows that the variability of the cluster assignments and the clusters themselves could be producing results different from the optimal clustering, and through multiple runs results have changed. 8 clusters tends to minimize the inertia, while the silhouette score tends to dip at 4 and increase a small amount from then on.

In [ ]:
# visualizing our clusters, cluster_number = 4
kmeans_final = cs.MiniBatchKMeans(n_clusters = 8, batch_size = 100).fit(clusterdata)
clusterdata['labels'] = kmeans_final.labels_

# examining in 2D
# 'acousticness', 'danceability', 'duration_ms', 'energy',
# 'instrumentalness', 'liveness', 'loudness', 'speechiness',
# 'tempo', 'valence', 'popularity'
sns.lmplot(x='energy', y='duration_ms', data=clusterdata, hue='labels', fit_reg=False, height = 6, aspect = 2)
sns.lmplot(x='popularity', y='duration_ms', data=clusterdata, hue='labels', fit_reg=False, height = 6, aspect = 2)
sns.lmplot(x='energy', y='loudness', data=clusterdata, hue='labels', fit_reg=False, height = 6, aspect = 2)

When examining our optimal cluster fit, there is a noticeable cluster divide on many 2-D slices involving the duration_ms column. This could possibly hint that this data feature is an important one in splitting the data, and could play a role in giving good recommendations. 

However, seeing as some of the columns were more correlated than others in our heatmap, such as loudness, energy, and tempo, it could be that clusters do not emerge between other features that'd probably play a bigger role in determining whether a song's fit for recommendation or not because of their overlap. Moving forward, PCA could be a good idea in assisting in assigning better clusters, but with any dimensionality reduction technique, some interpretability is sacrificed as a trade off.

Another good idea is to see whether individual columns tends to cluster as dramatically as duration_ms does. For example, are songs of a certain loudness, danceability, etc., clustered together?

**RECOMMENDER SYSTEM**

From here we start building our recommender system. Since there are no users within this dataset, this system will rely strictly on similarity between any two song's characteristics. We investigated these earlier utilizing a some basic data exploratory, a correlation heatmap, and cluster analysis.

As such, the content-based filtering approach based on feature similarity will work best for us, which we will compose as a class and package all of its features and functions within this framework.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

class SpotifyRecommender(object):
    
    def __init__(self, max_num = 5):
        if max_num > 30:
            print('No larger than 30 similar songs should be recommended at one time; value set to thirty')
            max_num = 30
        elif max_num <= 0:
            print('Have to recommend at least one song... value set to 5')
            max_num = 5
        self.max_num_recommendations = max_num
        self.recommendations = {}
    
    def train(self, data):
        '''
        Trains our recommender system using the data provided in the function call
        
        We take in a data matrix of all the song data we'd wish to utilize when training,
        create an index of each song's most similar songs within our dataset based on scikit-learn's
        cosine similarity metric, and then store this for each song id.
        
        RETURN: Nothing
        '''
        # maybe use linear_kernel?
        self.data = data
        
        # compute a similarity matrix for all of the data points to all other data points
        similarity = cosine_similarity(self.data[['acousticness', 'danceability', 'duration_ms', 'energy',
                         'instrumentalness', 'liveness', 'loudness', 'speechiness',
                         'tempo', 'valence', 'popularity']])
        
        # iterate through the rows of data
        for idx, row in self.data.iterrows():
            
            # get the most similar indices to the current row of our similarity matrix, 
            # remove first entry because it's most similar
            similar_indices = similarity[idx].argsort()[:-1*(self.max_num_recommendations+1):-1]
            # debug statement
            # print(len(similar_indices))
            
            # index into data frame and append a list of (cosine similarity score, similar track id) pairs
            # to the song with track_id row[id] in our dictionary
            similar_tracks = [(similarity[idx][i], data['id'][i]) 
                              for i in similar_indices]
            
            # append recommendations to dictionary by song id
            self.recommendations[row['id']] = similar_tracks[1:]
            
    
    def retrieve(self, track_id = None):
        '''
        This function does all of the retrieval work of a song's more involved details so that
        recommendations can be done simply by an id being passed
        
        RETURN: Artist and Name contained within row of provided dataframe indexed at track_id
        '''
        temp = self.data.loc[self.data["id"] == track_id][['artists', 'name']]
        artists = temp['artists']
        name = temp['name']
        return [artists, name]
    
    
    def predict(self, track_id = None):
        '''
        Simply put... just "predict" (i.e. return) the most similar songs
        
        Given a track_id, output the (max_num_recommendations) most similar songs in our dataset
        
        RETURN: The (max_num_recommendations) most similar tracks for given (track_id)
        '''
        # get song's data
        song = self.retrieve(track_id)
        
        # print
        print("Retrieving song recommendations similar to " + song[1] + " by " + song[0] + '...')
        
        # for entry brought up in our recommendations table
        for recommended_song in self.recommendations[track_id]:
            print(self.retrieve(recommended_song[1])[0])
            print(self.retrieve(recommended_song[1])[1])
            print("Recommended song: ID: " + str(recommended_song[1]) + " Name: " + self.retrieve(recommended_song[1])[1] 
                  + " by Artist: " + self.retrieve(recommended_song[1])[0] +  " . Similarity score: " + str(recommended_song[0]))
        

In [ ]:
recommender = SpotifyRecommender(5)
recommender.train(tracks.iloc[:100])
recommender.predict(tracks.iloc[0]['id'])